In [4]:
import overpy
import shapefile
import urllib.request
import time
import shapely.geometry
import fiona
from fiona.crs import from_epsg

In [5]:
inshp = "area/reg_buff.shp"
sf = shapefile.Reader(inshp)
bbox = sf.bbox

In [9]:
def convert_int(x):
    try:
        return int(x)
    except ValueError:
        return x
    
    
def get_osm(tag):
    api = overpy.Overpass()
    while True:
        try:
            res = api.query("""
                way({},{},{},{}) ["{}"];
                (._;>;);
                out body;
                """.format(bbox[1], bbox[0], bbox[3], bbox[2], tag))
            return res
        except:
            time.sleep(4)                   #time delay for the next query 4 seconds in case of many requests
    

def define_prj(name):
    #add a projection file
    prj = urllib.request.urlopen("http://spatialreference.org/ref/epsg/4326/esriwkt/").read()
    with open(name + '.prj', "w") as f:
        f.write(prj.decode('utf-8'))
        
        
def export_shp(vals, schema, name):
    #filter out different geometry types in the result request
    obj_dict = {shapefile.POINT: [[], []], shapefile.POLYGON: [[], []], shapefile.POLYLINE: [[], []]}
    for val in vals:
        crd_set = []
        for crd in val[0]:
            if crd not in crd_set:
                crd_set.append(crd)
        if len(val[0]) == 1:
            for i in [0,1]:
                obj_dict[shapefile.POINT][i].append(val[i])
        elif len(val[0]) != len(crd_set):
            for i in [0,1]:
                obj_dict[shapefile.POLYGON][i].append(val[i])
        else:
            for i in [0,1]:
                obj_dict[shapefile.POLYLINE][i].append(val[i])
    
    #export to shapefile
    for f in schema:
        f[0] = f[0][:10]      #shorten field names to 10 characters
    schema[0][0] = 'type'     #rename the first field to 'type'
    for k in obj_dict.keys():
        if obj_dict[k][0]:    #if the geom type has values for this osm tag
            w = shapefile.Writer(shapeType=k)
            w.fields = schema
            #w.records.extend(obj_dict[k][1])
            if k == 1: #point
                for i in range(len(obj_dict[k][0])):
                    w.point(*obj_dict[k][0][i])
                    w.record(*obj_dict[k][1][i])
            else:
                for i in range(len(obj_dict[k][0])):
                    w.poly(parts=[obj_dict[k][0][i]], shapeType=k)
                    w.record(*obj_dict[k][1][i])
            w.save(name + str(k))
            define_prj(name + str(k))

In [17]:
def parse_osm(obj_name, schema, additional_tags):
    #main func
    #download osm data with overpass api
    result = get_osm(obj_name)
    additional_result = []
    for atag in additional_tags:
         additional_result.append(get_osm(atag))
               
    #parse osm data
    schema_fields = [j[0] for j in schema]
    additional_coords = [[[[float(node.lon), float(node.lat)] for node in way.nodes] for way in res.ways] for res in additional_result]
    params = [list(map(convert_int, [way.tags.get(f, ' ') for f in schema_fields])) for way in result.ways]
    coords = [[[float(node.lon), float(node.lat)] for node in way.nodes] for way in result.ways]  
    if additional_tags:
        for i, acrd in enumerate(additional_coords):
            type_val = additional_tags[i].rstrip()
            for j, elem in enumerate(acrd):
                amenity_val = additional_result[i].ways[j].tags.get(additional_tags[i], '').rstrip()
                try:                                     #if there is an object with the same geometry in the original tag, update the values
                    k = coords.index(elem)
                    params[k][0] = type_val
                    params[k][2] = amenity_val
                    acrd.remove(elem)
                except ValueError:                       #if no object with the same geometry found, add the object
                    coords.append(elem)
                    prm_new = list(map(convert_int, [additional_result[i].ways[j].tags.get(f, '') for f in schema_fields]))
                    prm_new[0] = type_val
                    prm_new[2] = amenity_val
                    params.append(prm_new)
    del result, additional_result
    
    vals = list(zip(coords, params))
    export_shp(vals, schema, obj_name)
    

In [24]:
objects = [['building', [['building','C',30,0],['name','C',254,0],['amenity','C',50,0]], ['shop', 'tourism']],\
           ['boundary', [['boundary','C',30,0],['name','C',254,0],['admin_level','N',0,0],['place','C',50,0],['koatuu','C',20,0],['population','N',0,0]], []],\
            ['natural', [['natural','C',30,0],['name','C',254,0],['amenity','C',50,0]], ['geological', 'landcover', 'waterway']],\
            ['landuse', [['landuse','C',30,0],['name','C',254,0],['amenity','C',50,0]], ['aeroway', 'leisure', 'tourism']],\
            ['highway', [['highway','C',30,0],['name','C',254,0],['amenity','C',50,0], ['int_ref','C',254,0],['surface','C',50,0]], ['railway', 'trafic_sign','public_transport']],\
            ['man_made', [['man_made','C',30,0],['name','C',254,0],['amenity','C',50,0]], ['power', 'office', 'shop', 'leisure', 'tourism', 'historic']],\
           ['place', [['place','C',30,0],['name','C',254,0],['koatuu','C',20,0],['population','N',0,0]], []],\
           ['natural', [['natural','C',30,0],['name','C',254,0],['amenity','C',50,0]], ['geological', 'landcover']],\
           ]

In [ ]:
for tag in objects:
    parse_osm(tag[0], tag[1], tag[2])

In [ ]:
# POLYGONS
# building=*
#заменять значения в аменити если что-то есть
# If the object has also tag 'shop=*' or 'tourism=*', key (e.g. 'shop') has to be written in 'type' and value (e.g. 'convenience') has to be written in 'amenity'


# natural=*
# If the object has also tag 'geological=*' or 'landcover=*', key (e.g. 'landcover') has to be written in 'type' and value (e.g. 'grass') has to be written in 'amenity'


# landuse=*
# Includes also objects with tags 'aeroway=*', 'leisure=*' or 'tourism=*' (if it is not a building), where key (e.g. 'aeroway') stores in 'type' and its value stores in 'amenity'



# LINES
# highway=*
# Includes also objects with tags 'railway=*', where key (e.g. 'railway') stores in 'type' and its value stores in 'amenity'

# natural=*
# Includes also objects with tags 'waterway=*', where key (e.g. 'waterway') stores in 'type' and its value stores in 'amenity'

# man_made=*
# Includes also objects with tags power=*', where key (e.g. 'power') stores in 'type' and its value stores in 'amenity'


# POINTS
# highway=*
# Includes also objects with tags 'trafic_sign=*', railway=*' or 'public_transport=*', where key (e.g. 'public_transport') stores in 'type' and its value stores in 'amenity'


# natural=*
# If the object has also tag 'geological=*' or 'landcover=*', key ('landcover', for instance) has to be written in 'type' and value ('grass', for instance) has to be written in 'amenity'


# man_made=*
# Includes also objects with tags 'historic=*', office=*', 'shop=*', power=*', 'leisure=*' or 'tourism=*', where key (e.g. 'tourism') stores in 'type' and its value stores in 'amenity'